In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0,'content/drive/MyDrive/MotorImagery-master')

import IPython
from os import system
%clear

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import os
import numpy as np


def EEGNet(nb_classes = 4, Chans = 32, Samples = 1024, 
             dropoutRate = 0.5, kernLength = 32, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version....
    
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal #possibly decrease to 1
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (1, Chans, Samples))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (1, Chans, Samples),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)
  
  
  
  
  # need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   


def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25    
    
    Note that this implementation has not been verified by the original 
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations. 
    """

    # start the model
    input_main   = Input((1, Chans, Samples))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(1, Chans, Samples),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

In [ ]:
# CollectedData TrainEEG
import numpy as np
import h5py
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

# channels first
tf.keras.backend.set_image_data_format('channels_first')

# establish variables
subject_id = [1]
fs = 256                            # sampling rate 256Hz
num_classes = 4                     # four classes
num_channels = 32                    # 8 channels
tot_samples = 1024                  # 1000 total time points
window = [500]                 # number of samples in window we're looking at

# Modify this value for windowed
num_samples = 500


output_path = '/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet/500Windowed/0/EEG_Train'
data_path = '/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet/500Windowed/0'

if not os.path.exists(output_path):
    os.mkdir(output_path)
    print('created %s',output_path)

##########
# Train regular
##########
for n in subject_id:
    print()
    print("Subject: ", n)
    print()
      
   
    # Load data
    y_train = np.load(data_path+"/y_train_"+str(n)+".npy")
    X_train = np.load(data_path+"/X_train_"+str(n)+".npy")

    #print("y train, size:", len(y_train))
    #print(y_train)


    print("x train, shape:", X_train.shape)


    trials, dim1, dim2 = X_train.shape

    print(trials)
    print(dim1)
    print(dim2)
  


    if (dim1 == num_channels):
        # chans = dim1, samples = dim2
        X_train = np.reshape(X_train, (trials, 1 , dim1, dim2))
    elif (dim1 == num_samples):
        # samples = dim1, chans = dim2
        X_train = np.transpose(X_train, (0, 2, 1))
        X_train = np.reshape(X_train, (trials, 1, dim2, dim1))
        X_train= np.array(X_train)
    else:
        print("Error")
    
    y_val = np.load(data_path+"/y_val_"+str(n)+".npy")
    X_val = np.load(data_path+"/X_val_"+str(n)+".npy")
    trials, dim1, dim2 = X_val.shape
    if (dim1 == num_channels):
        # chans = dim1, samples = dim2
        X_val = np.reshape(X_val, (trials, 1 , dim1, dim2))
    elif (dim1 == num_samples):
        # samples = dim1, chans = dim2
        X_val = np.transpose(X_val, (0, 2, 1))
        X_val = np.reshape(X_val, (trials, 1, dim2, dim1))
    else:
        print("Error")

    # Build Model
    model = EEGNet(Samples = num_samples)
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
    model.summary()
    # Add Checkpoint
    checkpointer = ModelCheckpoint(filepath= output_path+"/subject"+str(n), 
                                   monitor='val_acc',
                                   mode='max',
                                   verbose=1,
                                   save_best_only=True)
    # Train and save model
    print("X_train: ", X_train.shape)
    print("y_train: ", y_train.shape)
    print("X_val: ", X_val.shape)
    print("y_val: ", y_val.shape)
    #print(y_val[0:9])
    history = model.fit(X_train,
                         y_train,
                         batch_size=16,
                         validation_data=(X_val, y_val),
                         epochs=500,
                         callbacks=[checkpointer])
    history.history.keys()
    history.history

    # labels, predictions, num_classes=None, weights=None, dtype=tf.dtypes.int32,
    # name=None)

    np.save(output_path+"/subject"+str(n)+"_acc", history.history["acc"])
    np.save(output_path+"/subject"+str(n)+"_loss", history.history["loss"])
    np.save(output_path+"/subject"+str(n)+"_val_acc", history.history["val_acc"])
    np.save(output_path+"/subject"+str(n)+"_val_loss", history.history["val_loss"])
      
    print()
    print()
    
print()
print()

In [ ]:
# TestEEG
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt



# channels first
tf.keras.backend.set_image_data_format('channels_first')


# Define variables
subject_id = [1]      
num_channels = 32                 # 8 channels
tot_samples = 1000                # 1000 total samples 
fs = 250                          # sampling rate 250Hz

# define file paths

data_path = '/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet/500Windowed/0'
weights_path = '/content/drive/MyDrive/MotorImagery-master/Data/CollectedData/ArdoWet/500Windowed/0/EEG_Train'
num_samples = 500
accuracy=[]



# loop through subjects 
for n in subject_id:
    # load testing data
    X_test = np.load(data_path+"/X_test_"+str(n)+".npy")
    y_test = np.load(data_path+"/y_test_"+str(n)+".npy")
    # Reshape Xtest data into (trials, 1, chans, samples)
    trials, dim1, dim2 = X_test.shape
    if (dim1 == num_channels):
        # chans = dim1, samples = dim2
        X_test = np.reshape(X_test, (trials, 1 , dim1, dim2))
    elif (dim1 == num_samples):
        # samples = dim1, chans = dim2
        X_test = np.transpose(X_test, (0, 2, 1))
        X_test = np.reshape(X_test, (trials, 1, dim2, dim1))
    else:
        print("Error")
        
    # Build Model
    model = EEGNet(Samples = num_samples)
    model.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    
    # Load trained model
    model.load_weights(weights_path+"/subject"+str(n))
            
    # Evaluate
    loss, acc = model.evaluate(X_test, y_test, verbose=1)
    pred_array = model.predict(X_test)
    
    accuracy.append(acc)
    print("Subject ", n, ": {:5.2f}%".format(100*acc))
    print()

print()
print("Mean Accuracy of EEGNet is",np.mean(accuracy)*100,"%")
print("Standard Deviation of EEGNet is",np.std(accuracy))

2/2 [==============================] - 0s 7ms/step - loss: 1.1194 - accuracy: 0.4500
Subject  1 : 45.00%


Mean Accuracy of EEGNet is 44.999998807907104 %
Standard Deviation of EEGNet is 0.0


In [ ]:
from sklearn.metrics import confusion_matrix


predictions = np.zeros(len(pred_array[:,0]))
for i in range(len(pred_array[:,0])):
  predictions[i] = np.argmax(pred_array[i])

cm = confusion_matrix(np.ravel(y_test), predictions)
tasks = ['left','right','down','up']
print(cm)

[[0 9 3 0]
 [4 5 1 2]
 [1 4 2 0]
 [1 4 0 4]]


In [ ]:
#4-class system
predictions = np.zeros(len(pred_array[:,0]))
for i in range(len(pred_array[:,0])):
  predictions[i] = np.argmax(pred_array[i])

print(' ')
print('The order of predicted labels is:')
print(predictions)

#predictions2 = np.where(predictions == 0, 1 , predictions)
#print(predictions2)

#Alphabet table
table = { '[0. 0. 0.]': 'A', '[0. 0. 1.]': 'B', '[0. 0. 2.]': 'C', '[0. 1. 0.]': 'D', '[0. 1. 1.]': 'E', '[0. 1. 2.]': 'F',
          '[0. 2. 0.]': 'G', '[0. 2. 1.]': 'H', '[0. 2. 2.]': 'I', '[1. 0. 0.]': 'J', '[1. 0. 1.]': 'K', '[1. 0. 2.]': 'L',
          '[1. 1. 0.]': 'M', '[1. 1. 1.]': 'N', '[1. 1. 2.]': 'O', '[1. 2. 0.]': 'P', '[1. 2. 1.]': 'Q', '[1. 2. 2.]': 'R',
          '[2. 0. 0.]': 'S', '[2. 0. 1.]': 'T', '[2. 0. 2.]': 'U', '[2. 1. 0.]': 'V', '[2. 1. 1.]': 'W', '[2. 1. 2.]': 'X',
          '[2. 2. 0.]': 'Y', '[2. 2. 1.]': 'Z', '[2. 2. 2.]': ' ', '[3. 0. 0.]': '1', '[3. 0. 1.]': '2', '[3. 0. 2.]': '3',
          '[3. 1. 0.]': '4', '[3. 1. 1.]': '5', '[3. 1. 2.]': '6', '[3. 2. 0.]': '7', '[3. 2. 1.]': '8', '[3. 2. 2.]': '9',
}
print(' ')
print('The order of viable codons is:')
letter = ""
for i in range(0, len(predictions)- len(predictions)%3,3):
    codon = predictions[i:i + 3]
    if codon[1] == 3 or codon[2] == 3:
      continue
    print(codon)
    letter += table[str(codon)]
print(' ')
print('The predicted text is:')
print(letter)

 
The order of predicted labels is:
[3. 3. 2. 3. 2. 0. 3. 1. 1. 0. 0. 0. 2. 0. 0. 1. 0. 0. 2. 0. 0. 2. 3. 1.
 3. 2. 0. 1. 0. 1. 2. 3. 0. 2. 3. 3. 2. 1. 2. 3. 1. 2. 0. 2. 3. 0. 3. 2.]
 
The order of viable codons is:
[3. 2. 0.]
[3. 1. 1.]
[0. 0. 0.]
[2. 0. 0.]
[1. 0. 0.]
[2. 0. 0.]
[3. 2. 0.]
[1. 0. 1.]
[2. 1. 2.]
[3. 1. 2.]
 
The predicted text is:
75ASJS7KX6
